In [323]:
import pyxdf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler

In [324]:
channels = ['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7'
            , 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz'
            , 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2', 'ACC_X', 'ACC_Y', 'ACC_Z']
removed_channels = ['Fp1', 'F8', 'F7', 'Fp2', 'F3', 'F4']
# TODO ADD AUX WHEN CONNECTED
columns = channels
frequencies = ['8.18', '9', '10', '11.25', '12.85', '15']


data, header = pyxdf.load_xdf('../data/BCISpellerV1/SpellerV1_15_1/dejittered-shited100ms-decimate2-eeg-and-dejittered-unity-freq.xdf')


In [325]:

def plot_single(df,df_ts, column):
    axis = plt.subplot()
    axis.plot(df_ts['time'], df[column])
    axis.set_title(column)
    plt.show()

def plot_eeg(df, df_ts):
    df.columns = columns
    index = 0
    for i in range(32):
        color = 'blue'
        index += 1
        if channels[i] in removed_channels:
            color = 'red'

        axis = plt.subplot()
        axis.plot(df_ts['time'], df[channels[i]], color=color)
        axis.set_title(channels[i])
        plt.show()





In [326]:
def load_frequencies(path = '../data/ERPSpellerDataV1/ERPSpellerDataA30Sec8-18hz'):
    data, header = pyxdf.load_xdf(path+'/dejittered-unity-frequencies-Yformat.xdf')
    df_stream = None
    df_stream_ts = None
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = frequencies
    return df_stream, df_stream_ts

def load_eeg(path = '../data/ERPSpellerDataV1/ERPSpellerDataA30Sec8-18hz'):
    data, header = pyxdf.load_xdf(path+'/dejittered-eeg.xdf')
    df_stream = None
    df_stream_ts = None
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = channels
    return df_stream, df_stream_ts

def load_eeg_and_freq(path = '../data/ERPSpellerDataV1/ERPSpellerDataA30Sec8-18hz'):
    data, header = pyxdf.load_xdf(path+'/dejittered-shited100ms-decimate2-eeg-and-dejittered-unity-freq.xdf')
    df_stream = None
    df_stream_ts = None
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = frequencies+ channels
    return df_stream, df_stream_ts

In [327]:
flip = False
path = '../data/BCISpellerV1/SpellerV1_8.18_5/'
df_eeg, df_ts_eeg = load_eeg(path)
df_freq, df_ts_freq = load_frequencies(path)
df_eeg_freq, df_ts_eeg_freq = load_eeg_and_freq(path)
chosen_channel = 'Oz'

start = 250*3
end = 250*4
print(len(df_eeg_freq))
#plot_single(df_eeg_freq[start:end], df_ts_eeg_freq[start:end], chosen_channel)
#plot_single(df_eeg_freq[start:end], df_ts_eeg_freq[start:end], '8.18')

X = df_eeg_freq[start:end][[chosen_channel, 'O1', 'O2', 'P3', 'P4', 'Pz']]


for i in frequencies:
    Y = df_eeg_freq[start:end][i]
    ca = CCA(n_components=1)
    ca.fit(X, Y)
    X_c, Y_c = ca.transform(X, Y)
    print(i)
    print(np.corrcoef(X_c[:, 0], Y_c[:, 0]))

2581
8.18
[[1.         0.17804908]
 [0.17804908 1.        ]]
9
[[1.         0.14596749]
 [0.14596749 1.        ]]
10
[[1.        0.1459319]
 [0.1459319 1.       ]]
11.25
[[1.         0.19823804]
 [0.19823804 1.        ]]
12.85
[[1.         0.09668205]
 [0.09668205 1.        ]]
15
[[1.         0.07074339]
 [0.07074339 1.        ]]
